# Library

In [1]:
import numpy as np
from selenium import webdriver
from time import sleep
import re
from pymongo import MongoClient
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import random
from selenium.common.exceptions import NoSuchElementException , ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options

# Class Crawl

In [2]:
class nhadat123():
    def __init__(self,link,name=[]):
        self.link = link
        self.name = name
        self.result = pd.DataFrame(columns=name)
        options= Options()
        options.page_load_strategy = 'eager'
        options.headless = True
        options.add_argument(r"--user-data-dir=C:\\Users\\USER\AppData\\Local\\Google\\Chrome\\User Data\Default")
        options.add_argument("--disable-javascript")
        options.add_argument("--blink-settings=imagesEnabled=false")
        self.driver = webdriver.Chrome(options=options)
    def Crawl(self):
        web_page=self.link
        while True :
            List_link , next_page = self.get_links(web_page)
            for i in range(len(List_link)):
                # if List_link[i] in link_have :
                #     continue
                self.getData(List_link[i])
            web_page = next_page
            if web_page == None:
                break
    def get_links(self,link):
        driver = self.driver
        driver.get(str(link))
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
            '.content-item .text .ct_title a'))
        )

        text = driver.find_elements(By.CSS_SELECTOR,
        '.content-item .text .ct_title a')
        links= [i.get_attribute('href') for i in text]
        current=int(driver.find_element(By.CSS_SELECTOR,'.page a.active').text)
        listNext_page = driver.find_elements(By.CSS_SELECTOR,'.page a')
        next_page=[x.get_attribute('href') for x in listNext_page][current]
        return links  ,next_page
    def getData(self,link):
        driver = self.driver
        test = pd.Series(index=self.name)
        driver.get(str(link))


        # Các nội dung tổng quan về mẫu
        test['Link'] = link
        test['Title'] = driver.find_element(By.CSS_SELECTOR,'h1').text
        test['Address'] = driver.find_element(By.CSS_SELECTOR,'.address .value').text
        test['Price'] = driver.find_element(By.CSS_SELECTOR,'.price').text
        listData=driver.find_elements(By.CSS_SELECTOR,'.moreinfor1 .infor table tr')
        data= [i.text for i in listData]
        for text in data:
            # Tìm Mã tin
            ma_tin = re.search(r'Mã tin (\d+)', text)
            if ma_tin:
                test["Mã tin"] = ma_tin.group(1)

            # Tìm Hướng
            huong = re.search(r'Hướng (\w+)(?: (\w+))? Phòng ăn', text)
            if huong:
                test["Hướng"] = huong.group(1)

            # Tìm Loại tin
            loai_tin = re.search(r'Loại tin ([\w\s]+) Lộ giới', text)
            if loai_tin:
                test["Loại tin"] = loai_tin.group(1)

            # Tìm Đường trước nhà
            duong_truoc_nha = re.search(r'Lộ giới (\d+m)', text)
            if duong_truoc_nha:
                test["Đường trước nhà"] = duong_truoc_nha.group(1)

            # Tìm Loại BDS
            loai_bds = re.search(r'Loại BDS ([\w\s]+) Pháp lý', text)
            if loai_bds:
                test["Loại BDS"] = loai_bds.group(1)

            # Tìm Pháp lý
            phap_ly = re.search(r'Pháp lý ([\w\s/]+) Sân thượng', text)
            if phap_ly:
                test["Pháp lý"] = phap_ly.group(1)

            # Tìm Chiều ngang
            chieu_ngang = re.search(r'Chiều ngang (\d+m)', text)
            if chieu_ngang:
                test["Chiều ngang"] = chieu_ngang.group(1)

            # Tìm Chiều dài
            chieu_dai = re.search(r'Chiều dài (\d+m)', text)
            if chieu_dai:
                test["Chiều dài"] = chieu_dai.group(1)

            # Tìm Số lầu
            so_lau = re.search(r'Số lầu (\d+)', text)
            if so_lau:
                test["Số lầu"] = so_lau.group(1)

            # Tìm Số phòng ngủ
            so_phong_ngu = re.search(r'Số phòng ngủ (\d+)', text)
            if so_phong_ngu:
                test["Số phòng ngủ"] = so_phong_ngu.group(1)
            
            dien_tich = re.search(r'Diện tích (\d+m)',text)
            if dien_tich:
                test['Square']= dien_tich.group(1)
        self.result = pd.concat([self.result, test.to_frame().T], ignore_index=True)


# Crawl

- setup info

In [3]:
link_web ='https://123nhadatviet.com/rao-vat/can-ban/nha-dat.html'
nameCol = ['Address','Square','Title','Mức giá','Link','Mã tin', 'Hướng', 'Loại tin', 'Đường trước nhà', 'Loại BDS', 'Pháp lý', 'Chiều ngang', 'Chiều dài', 'Số lầu', 'Số phòng ngủ']

In [4]:
X = nhadat123(link=link_web,name=nameCol)

There was an error managing chromedriver (error decoding response body); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


In [5]:
X.Crawl()

C:\Users\USER\AppData\Local\Temp\ipykernel_6004\2398968019.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://123nhadatviet.com/chinh-chu-nhuong-lai-toa-van-phong-hang-a-pho-duong-dinh-nghe-cau-giay-4500m2x26t-4612747.html' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test['Link'] = link
C:\Users\USER\AppData\Local\Temp\ipykernel_6004\2398968019.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://123nhadatviet.com/chinh-chu-can-ban-can-nha-van-phong-o-pho-phao-dai-lang-dong-da-75m2x6t-nhinh-hon-17-ty-3520846.html' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test['Link'] = link
C:\Users\USER\AppData\Local\Temp\ipykernel_6004\2398968019.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"h1"}
  (Session info: chrome=130.0.6723.117); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF662603B05+28005]
	(No symbol) [0x00007FF662568400]
	(No symbol) [0x00007FF66240580A]
	(No symbol) [0x00007FF662455A3E]
	(No symbol) [0x00007FF662455D2C]
	(No symbol) [0x00007FF66249EA97]
	(No symbol) [0x00007FF66247BA7F]
	(No symbol) [0x00007FF66249B8B3]
	(No symbol) [0x00007FF66247B7E3]
	(No symbol) [0x00007FF6624475C8]
	(No symbol) [0x00007FF662448731]
	GetHandleVerifier [0x00007FF6628F647D+3118813]
	GetHandleVerifier [0x00007FF662946CD0+3448624]
	GetHandleVerifier [0x00007FF66293CF4D+3408301]
	GetHandleVerifier [0x00007FF6626CA45B+841403]
	(No symbol) [0x00007FF66257345F]
	(No symbol) [0x00007FF66256F4D4]
	(No symbol) [0x00007FF66256F66D]
	(No 

In [6]:
X.result

,Address,Square,Title,Mức giá,Link,Mã tin,Hướng,Loại tin,Đường trước nhà,Loại BDS,Pháp lý,Chiều ngang,Chiều dài,Số lầu,Số phòng ngủ,Price
0,"Phố Dương Đình Nghệ, Phường Yên Hòa, Quận Cầu ...",4500m,Chính Chủ nhượng lại toà văn phòng hạng A phố ...,NaN,https://123nhadatviet.com/chinh-chu-nhuong-lai...,4612747,_,Cần bán,NaN,Văn phòng,Giấy tờ hợp lệ,NaN,NaN,26,24,173 tỷ
1,"Phố Pháo Đài Láng, Phường Láng Thượng, Quận ...",75m,Chính chủ cần bán căn nhà văn phòng ở Phố Pháo...,NaN,https://123nhadatviet.com/chinh-chu-can-ban-ca...,3520846,_,Cần bán,NaN,Nhà phố,Sổ hồng/ Sổ đỏ,NaN,NaN,7,10,"22,5 tỷ"
2,"Đường Quốc lộ 61, Thị trấn Minh Lương, Huyện C...",103m,"Bán nền sổ sẵn 103m2 , gần chợ .",NaN,https://123nhadatviet.com/ban-nen-so-san-103m2...,6871019,Đông,Cần bán,12m,NaN,Sổ hồng/ Sổ đỏ,5m,20m,NaN,NaN,450 triệu
3,"Đường Đỗ Quang, Phường Trung Hòa, Quận Cầu Giấ...",70m,Chính chủ bán nhà phân lô ngõ 26 Đỗ Quang Cầu ...,NaN,https://123nhadatviet.com/chinh-chu-ban-nha-ph...,6613659,_,Cần bán,NaN,Nhà riêng,Sổ hồng/ Sổ đỏ,NaN,NaN,7,7,35 tỷ
4,"Phố Thái Hà, Phường Trung Liệt, Quận Đống Đa,...",65m,Chính chủ cần bán căn nhà ở ngõ 151A Thái Hà Đ...,NaN,https://123nhadatviet.com/chinh-chu-can-ban-ca...,3395601,_,Cần bán,NaN,Nhà riêng,Sổ hồng/ Sổ đỏ,NaN,NaN,5,7,10 ngàn
5,"Phố Nguyễn Đình Hoàn, Phường Nghĩa Đô, Quận Cầ...",62m,Chính bán nhà mặt phố Kinh doanh đỉnh phố Nguy...,NaN,https://123nhadatviet.com/chinh-ban-nha-mat-ph...,4824998,_,Cần bán,NaN,Nhà phố,Sổ hồng/ Sổ đỏ,NaN,NaN,6,10,"22,6 tỷ"
6,"Đường Nguyễn Thị Định, Phường Trung Hòa, Quận ...",75m,Chính chủ bán nhà phân lô vỉa hè ngõ 30 Nguyễn...,NaN,https://123nhadatviet.com/chinh-chu-ban-nha-ph...,5746632,_,Cần bán,NaN,Nhà riêng,Sổ hồng/ Sổ đỏ,NaN,NaN,5,10,"23,7 tỷ"
7,"Phố Liễu Giai, Phường Liễu Giai, Quận Ba Đình,...",50m,CHỦ NHÀ MUỐN BÁN NHÀ LIỄU GIAI BA ĐÌNH 50M2-6 ...,NaN,https://123nhadatviet.com/chu-nha-muon-ban-nha...,6871018,_,Cần bán,6m,Nhà riêng,Sổ hồng/ Sổ đỏ,5m,NaN,6,5,14 tỷ
8,"Phố Nguyên Hồng, Phường Láng Hạ, Quận Đống Đ...",48m,Chính chủ bán nhà phân lô ngõ 47 Nguyên Hồng Đ...,NaN,https://123nhadatviet.com/chinh-chu-ban-nha-ph...,5694500,_,Cần bán,NaN,Nhà riêng,Sổ hồng/ Sổ đỏ,NaN,NaN,5,9,"14,4 tỷ"
9,"Đường Thịnh Quang, Phường Ngã Tư Sở, Quận Đố...",48m,"Bán nhà ngõ Thịnh Quang, DT 48m, 4 tầng, MT 5m...",NaN,https://123nhadatviet.com/ban-nha-ngo-6-vinh-p...,6437853,_,Cần bán,NaN,Nhà riêng,NaN,NaN,NaN,NaN,NaN,"8,5 tỷ"


# Store Data

In [8]:
X.result.to_csv('123nhadat.csv',index=False)

- MongoDB

In [ ]:
client = MongoClient("mongodb://localhost:27017/")
db = client["batdongsan"]
collection = db["123nhadat"]
data=X.result
data_dict = data.to_dict("records")
collection.insert_many(data_dict)

InsertManyResult([ObjectId('67317bda6ed766b3c16129ab'), ObjectId('67317bda6ed766b3c16129ac'), ObjectId('67317bda6ed766b3c16129ad'), ObjectId('67317bda6ed766b3c16129ae'), ObjectId('67317bda6ed766b3c16129af'), ObjectId('67317bda6ed766b3c16129b0'), ObjectId('67317bda6ed766b3c16129b1'), ObjectId('67317bda6ed766b3c16129b2'), ObjectId('67317bda6ed766b3c16129b3'), ObjectId('67317bda6ed766b3c16129b4'), ObjectId('67317bda6ed766b3c16129b5'), ObjectId('67317bda6ed766b3c16129b6'), ObjectId('67317bda6ed766b3c16129b7'), ObjectId('67317bda6ed766b3c16129b8'), ObjectId('67317bda6ed766b3c16129b9'), ObjectId('67317bda6ed766b3c16129ba'), ObjectId('67317bda6ed766b3c16129bb'), ObjectId('67317bda6ed766b3c16129bc'), ObjectId('67317bda6ed766b3c16129bd'), ObjectId('67317bda6ed766b3c16129be'), ObjectId('67317bda6ed766b3c16129bf'), ObjectId('67317bda6ed766b3c16129c0'), ObjectId('67317bda6ed766b3c16129c1'), ObjectId('67317bda6ed766b3c16129c2'), ObjectId('67317bda6ed766b3c16129c3'), ObjectId('67317bda6ed766b3c16129

- postgresSQL

In [ ]:
from sqlalchemy import create_engine
data = X.result
# Thông tin kết nối PostgreSQL
db_user = "postgres"
db_password = "ky1213141516" # <Your_PostgresSQL_Password>
db_host = "localhost"
db_port = "3000"
db_name = "postgres" #<Your_Database_Password>

# Tạo kết nối với PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Lưu DataFrame vào PostgreSQL (chọn tên bảng phù hợp)
data.to_sql('123nhadat', engine, if_exists='replace', index=False)

In [7]:
(X.result).to_csv('E:\DE_JOB\Presonal_Project\Pineline_ETL\CrawlData\\123nhadat\\123nhadat.csv')